In [ ]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="gemini-3-flash-preview",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

TypeError: create_agent() got an unexpected keyword argument 'api_key'

In [6]:
import os
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

gemini_api_key = os.getenv("GEMINI_API_KEY")
# Now you can use openai_api_key when initializing your Langchain model


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage # Import HumanMessage for agent.invoke

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=gemini_api_key
)

agent = create_agent(
    model=llm, # Pass the instantiated LLM object
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
result = agent.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]}
)
result

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='79864c78-eaf1-4360-8279-e6f73d034ef6'), AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}, '__gemini_function_call_thought_signatures__': {'b97090d5-2a71-4b99-9574-016ed97ad648': 'EtkBCtYBAXLI2nwtGL6N/YdDXwyVnAkhHqXYcsc+JTsvxDNOZpYkM6mK7WutYHQzAlTYKAMkaGilH8xludVFLj9a7y7IGNLye4F6BGanJurL+HdcR3x6ShELkjh6SIG4swMImssToU2wI0jGm5QzTulPTKrEBiLxxBTDU4T2lLHiBBzwmYTI3f1gUJNWiKtrnQyJiEB2d5BTJsFGD5ckO7pHEaNG67g3LS1zpWjwv8f5XAuDAqxMjCLUoYv7r+yUjpyOc6gzncN3nSNeq7gFRM3k8eaHaV+daUn0rg=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b97f4-14da-75c3-8622-fa4f0e655885-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'b97090d5-2a71-4b99-9574-016ed97ad648', 'type': 'tool_call'}],

In [9]:
result

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='79864c78-eaf1-4360-8279-e6f73d034ef6'),
  AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}, '__gemini_function_call_thought_signatures__': {'b97090d5-2a71-4b99-9574-016ed97ad648': 'EtkBCtYBAXLI2nwtGL6N/YdDXwyVnAkhHqXYcsc+JTsvxDNOZpYkM6mK7WutYHQzAlTYKAMkaGilH8xludVFLj9a7y7IGNLye4F6BGanJurL+HdcR3x6ShELkjh6SIG4swMImssToU2wI0jGm5QzTulPTKrEBiLxxBTDU4T2lLHiBBzwmYTI3f1gUJNWiKtrnQyJiEB2d5BTJsFGD5ckO7pHEaNG67g3LS1zpWjwv8f5XAuDAqxMjCLUoYv7r+yUjpyOc6gzncN3nSNeq7gFRM3k8eaHaV+daUn0rg=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b97f4-14da-75c3-8622-fa4f0e655885-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'b97090d5-2a71-4b99-9574-016ed97ad648', 'type': 'tool_call'}

In [10]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [11]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [17]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gemini-3-flash-preview",
    model_provider="google_genai",  # <--- THIS IS CRITICAL
    temperature=0.5,
    timeout=10,
    max_tokens=1000
)
model

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-3-flash-preview', temperature=0.5, max_output_tokens=1000, timeout=10.0, client=<google.genai.client.Client object at 0x113b7dd10>, default_metadata=(), model_kwargs={})

In [18]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [19]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [47]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="I'm 'afraid' I don't have a 'latte' information on the cafe's location, but I can tell you it's 'brewing' up a beautiful day outside! It's still 'sun-sational' in Florida, so you won't need an 'umbrella-ccino' for your walk!", weather_conditions='Sunny')
ResponseFormat(punny_response="You're 'snow' welcome! I'm always happy to 'shower' you with information. Have a 'bolt' of a good time!", weather_conditions=None)


In [35]:
@dataclass
class Student:
    name: str
    age: int

Student(name="nureke", age=4)

Student(name='nureke', age=4)

In [37]:
@tool
def get_user_name(user_id: str) -> str:
    """returns the user name by user_id"""
    if user_id == "1":
        return "the user name is Aisha"
    else:
        return "the user name is Dimash"



In [38]:
agent = create_agent(
    model=model,
    # system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_user_name, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is my name?"}]},
    config=config,
    context=Context(user_id="1")
)


# Agents

In [41]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Error 404"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"


In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", api_key=gemini_api_key)
advanced_model = ChatGoogleGenerativeAI(model="gemini-3-pro-preview", api_key=gemini_api_key)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 30:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

agent = create_agent(
    model=basic_model,  # Default model
    tools=[search, get_weather, get_user_location, get_weather_for_location],
    middleware=[dynamic_model_selection],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "3"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "where is the cafe?"}]},
    config=config,
    context=Context(user_id="3")
)
response

{'messages': [HumanMessage(content='where is the cafe?', additional_kwargs={}, response_metadata={}, id='cc39df6d-c669-4967-bd5b-c7008d997e4a'),
  AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_user_location', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'0711c02f-3830-45bf-b116-adc534fbf16d': 'EuoCCucCAXLI2nwakt+bnL1GMUO5Vgrh6MPnGUDavjYqr44vSMnZuvNwKpFyMlRKFFbIbzJBU8qPq+DNLgQ1/NeXbBeuYlclqHwB75BQvU2BCjpgREomV+Lp5k4Z7mD9qNCZz6pQWTkO/TIsVk5ynAXa2rjVnJLSMAotvWxkaPdY58+O0wwZKF6iRftUhKiKZdmJvKsC65Fci7tue/mmdptg6/RaZwPEnxYqJIpi8IhcmmUdh0ZKP8cTqxo8MfdF52L29VziNsI+PjSZtE9vzYFX+2m6uQoJqKa27FsZsNze9c0y6BHvuRLqxR2AmfN8hnhSPYs8iG1lhilAbOB+7ZiLH8PwUZY67WdLLDVfC3YfjK+tUSObrcQ/A01GMadTrxCfNyBBH5yjUBvhtK5+SdM2xBJGItx4HRZew29RLGdiaLQoLxUsSjMzHlHC2rwxWPxFFkb/5jOkGs72ugB53Exd+bNw2rnBffWIfwg='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9894-a

In [64]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model=model,
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)

In [67]:
config = {"configurable": {"thread_id": "4"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "where is the cafe?"}]},
    config=config,
    context=Context(user_id="3")
)
response

{'messages': [HumanMessage(content='where is the cafe?', additional_kwargs={}, response_metadata={}, id='3aaa8a66-3c31-43b4-900b-f047fe41121f'),
  AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_user_location', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'10860561-79ea-426c-b4f0-44008cadd8d2': 'ErcGCrQGAXLI2nwFiDRLIAjCw5OXLyEqALeV5wBjpZr4feRbG1QQw6A4vbUc7+E4wAberROYMqRU6zntZYyVz/g+i7pRgYKePJxmeGpBxznfYuj325O+MjPXPFF2zbuzpGJ5Et4J+EdtIaA8NqAvlTQydo/xt7Mwm7UMBxalZo794PMbUVG9cHHVCwYfPWKXLMi9Sk2lr4UJDdkg/8hW2X0CHKaP2WnB/90wPCEVi2utQfHcWPf85JqCye35HWMDbY1ANyHMjiLVlQQViDuFj8ZwYpwA6YGxw71ZY+8VejlY1oh8RSxcPT8rqHVa+2bhrZE4V/V+mCLf2rcielXYRvC4ypVBEQntNCV02xzuGz4Gsn3lTiW5FkSziKH8rw7JZwIEfDaJoZy/FUUW585LMLiDg1mxQ1KdWwYgXeEt/kjqPex5wX6wXCzfMFq0yXSM78XI+Rb7Xw+mIsxtMbatu7oC6kQmZgFbxUuuyvh14pzrV9mEHlUh6bMejsDBFzeQhLCzEEd++6CiOy5hTewb2ZKm6nyIu+6bqiHRVDK9QTlf1K+yLkLdB0fi2QsUJZFuDHGnlBJVGoGYRhv9oBRQQf23RrdR3BSPam0A1NVfdBlBME8uPSb/PBqhhnP0REz+FOVYWHq8KTiNv1F0

In [68]:
import requests
from langchain.tools import tool

@tool
def get_real_weather(city: str) -> str:
    """Fetch real weather data for a given city using OpenWeatherMap API."""
    api_key = "e3803a006cc1312498da05884bd41764"  # Ensure your API key is stored in the environment variables
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key, "units": "metric"}  # Fetch weather in Celsius

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an error for bad HTTP responses
        data = response.json()

        # Extract weather information
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        return f"The weather in {city} is currently {weather_description} with a temperature of {temperature}°C."
    except requests.exceptions.RequestException as e:
        return f"Failed to fetch weather data: {e}"
    except KeyError:
        return "Could not parse weather data. Please check the city name or try again later."

In [69]:
import requests
from langchain.tools import tool

@tool
def get_my_real_location() -> str:
    """Fetch the user's real location based on their public IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        response.raise_for_status()  # Raise an error for bad HTTP responses
        data = response.json()

        # Extract location information
        city = data.get("city", "Unknown")
        region = data.get("region", "Unknown")
        country = data.get("country", "Unknown")
        return f"{city}, {region}, {country}"
    except requests.exceptions.RequestException as e:
        return f"Failed to fetch location data: {e}"

In [ ]:
@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model=model,
    tools=[get_real_weather, get_my_real_location],
    middleware=[handle_tool_errors],
    system_prompt=SYSTEM_PROMPT,
)

In [71]:
config = {"configurable": {"thread_id": "5"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the current weather?"}]},
    config=config,
    context=Context(user_id="5")
)
response

{'messages': [HumanMessage(content='what is the current weather?', additional_kwargs={}, response_metadata={}, id='4fb65c77-0272-4e41-bc20-afda5add6930'),
  AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_my_real_location', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'dc3ea527-cc2d-4ff5-9180-84746440048d': 'EtcDCtQDAXLI2nzmbdTUWzHx6yWqeZkstQg5wfaPDkmcvII/wrdsLtN8i9d1Q/gRRDdkpMdv6hZka53HUTz9Q9M7vFdGxQhZYwIsoHyKEq/7oM8Y6nHy0HRdL8x6VF8aZDp+pQt9d8usbxCiaw/b2sz6zIhiuuhXEZYAdkJXnKaIph3i3jdbuPMgMlepfdyy9K9vxWhgUL0eCev9RJKRhfzh/DrZdP52S/7WLkQoR2pIMu2Wev1SN9PhPq9w90lN+mtYz63WAmZ3jcP2OjjJPmG4eyd1gayk1l31uiNuAsItzroPmRVu+Qn6pBZxoTBpr3tPVjxmef7QRdntKOXVtPg99CZIm41vEeFRPt15WXJiWi91KkauJArmEJS7zo4zgVn5cCHejhiVC33PnNhNOKYMlQR8+LHV6MJ3c125vU8ehLHx7ax02NuHY8vTgT2X1eCuIYClauI/jCUcokqiFSzVjZsoZibHQJkNTjZZDx7xfjL0uTyieCtbcgszLlToXXL5ISDS4LL5GJSJj2NCguc8N+aLLaZxnk5IjUdb6GkJdBv/whhcEbUhSzYrBEs/wvHPO+XP2rY0YpNqBkvKu3woh0ROi633WlmxQ1kNAmrr9UKXKdD7EtMf'}}, respon

In [72]:
tools = [get_real_weather, get_my_real_location]

agent = create_agent(
    model,
    tools,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)


In [73]:
from langchain.agents import create_agent
from langchain.messages import SystemMessage, HumanMessage

literary_agent = create_agent(
    model=model,
    system_prompt=SystemMessage(
        content=[
            {
                "type": "text",
                "text": "You are an AI assistant tasked with analyzing literary works.",
            },
            {
                "type": "text",
                "text": "<the entire contents of 'Pride and Prejudice'>",
                "cache_control": {"type": "ephemeral"}
            }
        ]
    )
)

result = literary_agent.invoke(
    {"messages": [HumanMessage("Analyze the major themes in 'Pride and Prejudice'.")]}
)

In [74]:
result

{'messages': [HumanMessage(content="Analyze the major themes in 'Pride and Prejudice'.", additional_kwargs={}, response_metadata={}, id='6154a1c4-a376-4c88-a7d6-af51c9259341'),
  AIMessage(content=[{'type': 'text', 'text': 'Jane Austen’s *Pride and Prejudice* is a masterclass in social commentary, utilizing a sharp, ironic wit to dissect the manners and morals of the British Regency-era gentry. While the plot centers on the turbulent courtship between Elizabeth Bennet and Fitzwilliam Darcy, the novel explores profound themes regarding the human condition and societal structures.\n\nHere is an analysis of the major themes in the work:\n\n### 1. Love and Marriage\nAt its core, the novel is a study of the different motivations behind marriage. Austen presents a spectrum of unions to critique the economic and social pressures of her time:\n*   **Marriage for Security (Charlotte Lucas and Mr. Collins):** Charlotte’s marriage represents the grim reality for women of the era. Without a fortun

In [77]:
@tool
def web_search(query: str) -> str:
    """Perform a web search using the agent's search tool."""
    response = agent.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config=config,
        context=Context(user_id="1")
    )
    return response['messages'][-1].content

# Example usag

In [80]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt

agent = create_agent(
    model=model,
    tools=[web_search],
    middleware=[user_role_prompt],
    context_schema=Context
)

# The system prompt will be set dynamically based on context
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what the current date?"}]},
    context={"user_role": "expert"}
)
result


{'messages': [HumanMessage(content='what the current date?', additional_kwargs={}, response_metadata={}, id='31c020c3-8df9-4f55-aed6-976318476c23'),
  AIMessage(content=[{'type': 'text', 'text': 'The current date is **Wednesday, May 22, 2024**.', 'extras': {'signature': 'Es8CCswCAXLI2nwlQiQ3qno7a3IXvjhFBZB5soRTN4rvGe7u/16aMsOU2sLTDK2QvGeMsSS9qtp8u5vQdLwp7uuJeuPbKdoumUCGDWGTDBM9EM9zKdjYioYmbuvw7D5rZkFhXk1jY4T9bZJiuZiZ/qBCHx41qOvWaSaxyYXLDQwOgUj3Fx/WSfSBkJ7v4DFbc36ILJDfwSBYlMLe5LwjnbfaOlRHta+tI0+++tl/yO9IXuO9lp1PbI82fCXQ4qP0cB7T5h7M06tb5kXR6VoYrlw9Q86I9addcQJZwXxIBWhZ6O2V4edhdNQ0T3A6fp7sLYb6guHqXYfwgyc16mfWbjevNJN4E3qT6Duk8Rr0P8gefvCiZDLgxNpwpzHJjXoCR2eRNQjpSkkVW3rS1KUc7P5JZRG20ruPbp3CnPsp0qmXvDT68ksLRTYUiZeDWP2J4Rc='}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b98f8-ae0a-7350-8580-e4a7f4a36683-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={

{'messages': [HumanMessage(content='Explain machine learning', additional_kwargs={}, response_metadata={}, id='3fb0f80a-87be-490e-b3a5-badfd88f2e88'),
  AIMessage(content=[{'type': 'text', 'text': 'Machine Learning (ML) is a subfield of Artificial Intelligence (AI) focused on building systems that learn from data to perform tasks without being explicitly programmed for every possible scenario.\n\nIn traditional programming, a developer writes specific rules (logic) to process data and produce an output. In machine learning, the process is inverted: the computer is provided with data and the desired output, and it uses statistical algorithms to "learn" the underlying rules or patterns.\n\n---\n\n### 1. The Core Paradigms of Machine Learning\n\nMachine learning is generally categorized into three primary types based on how the algorithm learns:\n\n#### A. Supervised Learning\nThe model is trained on a **labeled dataset**, meaning the input data is paired with the correct answer. The goal